## Import modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
    
from sklearn.linear_model import RidgeCV # maybe used for GCV, as per the documentation

## Algorithm

**Input**: 

- S = Solution snaptshot matrix S (NxNs)
- e_s = Tolerance
- zeta = Correction factor
- param = Regularization parameter alpha* or parameter w
- u_ref = Reference solution

In [2]:
# Loads the FOM Snapshots as the matrix S
S = np.load('fom_snapshots.npy')

In [3]:
S.shape
# N = 9216
# Ns = 450

(9216, 450)

In [4]:
N = S.shape[0] 
Ns = S.shape[1]

In [5]:
e_s = 1e-4 # used to get a somewhat lower n_tra. Tried 1e-7, but then Ns > n(n+1)/2 wouldn't verify

In [6]:
zeta = 0.15 # following the paper

In [7]:
omega = 0.1 # again following the paper. Is this a guess?

In [8]:
u_ref = 0 # i imagine a referece solution would be u(0) = 0, is this the case?

In [43]:
# Thin SVD of S
US, SigmaS, YsT = np.linalg.svd(S, full_matrices=False)

In [44]:
US.shape

(9216, 450)

In [45]:
SigmaS.shape

(450,)

In [46]:
YsT.shape # Already Transposed ?

(450, 450)

In [47]:
# sets the initial value for sum_n = 0
sum_n = 0

# sets the value of sum_k as the sum of squared values of SigmaS vector
SigmaSSquared = np.square(SigmaS) 
sum_k = np.sum(SigmaSSquared)

print(sum_k)

2207187.5414926386


In [48]:
# Finds the mininum value of n that satisfies (4) 

for n in range (S.shape[1] - 1):
    sum_n += SigmaSSquared[n]
    if (sum_n/sum_k >= 1 - e_s):
        n_tra = n + 1
        break
        
print(n_tra)

12


In [49]:
# Create V, using only the first n_tra columns of US

V = US[:, :n_tra]

In [52]:
# Determines n_qua
n_qua = ((9 + 8*n_tra)**(1/2) - 3)/2
n_qua = (1+zeta)*n_qua

In [51]:
n_qua

4.166996690426768

In [61]:
# Determines n
n = min(n_qua, ((1+8*Ns)**(1.2)-1)/2)
n

4.166996690426768

In [62]:
# Rounds up n (is this correct?
n = int(np.ceil(n))
n

5

In [22]:
# i was considering V as the right basis matrix, now switched to the paper algorithm 
#V = np.load('RightBasisMatrix.npy')
#V.shape

In [80]:
# Truncates V, s.t. V.shape = N x n (9126,5)
# Maybe I should have used np.trunc ?

V = V[:, :n]
V.shape

(9216, 5)

In [82]:
# Declare E, Q_ and H_

E = np.empty((N,Ns))
Q_ = np.empty((int(n*(n+1)/2), Ns))
H_ = np.empty((N,int(n*(n+1)/2)))

In [67]:
print(E.shape)
print(Q_.shape)
print(H_.shape)

(9216, 450)
(15, 450)
(9216, 15)


In [76]:
# Populates E and Q_

for i in range(Ns):
    qi = np.dot(V.T, S[:,i]) # Transpose necessary?
    E[:,i] = S[:,i] - np.dot(V,qi) - u_ref # Transpose necessary?
    Q_[:,i] = np.unique(np.kron(qi,qi)) # Is this the correct method?

In [78]:
Q_.shape

(15, 450)

In [73]:
# Thin SVD of Q_

Uq_, Sigmaq_, Yq_ = np.linalg.svd(Q_, full_matrices=False)

In [84]:
# Checking if shapes have the correct shape
print(Uq_.shape)
print(Sigmaq_.shape)
print(Yq_.T.shape)

(15, 15)
(15,)
(450, 15)


In [30]:
# alpha_best not specified, so:

In [91]:
# Declares alpha_best and an empry array of size N
alpha_best = np.empty(N)

In [92]:
# Calculates nsamp

Nq = Sigmaq_.shape[0]
print(Nq)
nsamp = int(np.ceil(omega*Nq))
nsamp

15


2

In [94]:
# Creates and populates alpha_samp

alpha_samp = np.logspace(Sigmaq_[Nq-1], Sigmaq_[1], nsamp) # Not sure if this is the correct implementation uniformly distribution in log scale, from the maximum to the minimum singular value of Q_

In [95]:
alpha_samp.shape

(2,)

In [96]:
# Uses GCV (TO BE IMPLEMENTED) to determine the best regularization parameters

for i in range(N):
    G = np.empty(nsamp)
    for k in range(nsamp):
        G[k] = alpha_samp[k] #TODO: implement the GCV function
    alpha_best[i] = np.amin(G)

values, counts = np.unique(alpha_best[i], return_counts=True)
ind = np.argmax(counts)

alpha_star = values[ind] # GCV function not implemented

print(ind)
print(values)

0
[4.18906533e+27]


In [99]:
# Calculates h_i and inputs it to H_, line by line

for i in range(N):
    h_i = 0
    for l in range(Nq):
        h_i += ((Sigmaq_[l]**2)/(Sigmaq_[l]**2 + alpha_star**2))*np.dot((np.dot(Yq_[l].T,E[l,:].T)/Sigmaq_[l]),Uq_[l])
    H_[i,:] = h_i
    
print(H_.shape)
print(h_i.shape)
    

(9216, 15)
(15,)


In [100]:
print(H_)

[[-2.01538000e-58 -3.96296135e-57 -1.99923957e-56 ... -1.55046429e-54
  -8.13793179e-55  9.95870400e-55]
 [-2.01538000e-58 -3.96296135e-57 -1.99923957e-56 ... -1.55046429e-54
  -8.13793179e-55  9.95870400e-55]
 [-2.01538000e-58 -3.96296135e-57 -1.99923957e-56 ... -1.55046429e-54
  -8.13793179e-55  9.95870400e-55]
 ...
 [-2.01538000e-58 -3.96296135e-57 -1.99923957e-56 ... -1.55046429e-54
  -8.13793179e-55  9.95870400e-55]
 [-2.01538000e-58 -3.96296135e-57 -1.99923957e-56 ... -1.55046429e-54
  -8.13793179e-55  9.95870400e-55]
 [-2.01538000e-58 -3.96296135e-57 -1.99923957e-56 ... -1.55046429e-54
  -8.13793179e-55  9.95870400e-55]]
